In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from Data_Require import *
import plotly.graph_objs as go
from plotly import tools
from dash.dependencies import Input, Output
import plotly
from IPython.display import HTML
warnings.filterwarnings(action="ignore", message="unclosed",
                         category=ResourceWarning)

In [2]:
dat1 = pd.read_csv('Market.csv',index_col=0)
dat2 = pd.read_csv('Ab_ratio.csv',index_col=0)

In [11]:
Asset_Classes= ['SPY', 'EEM', 'EFA', 'EMB', 'IPE', 'DBC', 'GLD', 'USO', 'RWO', 'GII', 'DBV']
US_Sectors = ['IYC','IYE','IYF','IYH','IYJ','IYK','IYM','IYR','IYW','IYZ']
Developed_Countries_Stock = ['EWA', 'EWU', 'EWC', 'EWH', 'EWL', 'EWD', 'EWS', 'EWJ', 'EWG', 'EWI', 'EWP', 'EWQ']
Emerging_Markets_Stock = ['EWZ', 'EWW', 'EZA', 'EWY', 'GXC']
Fixed_Income = ['SHY', 'IEI', 'IEF', 'TLH', 'TLT', 'BWX', 'CRED', 'JNK', 'LQD']
FX= ['UUP', 'BZF', 'FXA', 'FXB', 'FXC', 'FXF', 'FXS', 'FXSG', 'FXY']

market_dict = {'Asset_Classes':Asset_Classes,
               'US_Sectors':US_Sectors,
               'Developed_Countries_Stock':Developed_Countries_Stock,
               'Emerging_Markets_Stock': Emerging_Markets_Stock,
               'Fixed_Income':Fixed_Income,
               'FX':FX}
asset_list = []
for key in list(market_dict.keys()):
    asset_list += market_dict[key]
today = dt.today().strftime("%Y-%m-%d")
start = '2000-01-01'
end = today

#dat1 = get_tb_dataframe(market_dict,start,end)
#dat2 = get_ar_dataframe(asset_list,start,end)

In [12]:
#correlation matrix for Asset Classes
corr,ht = correlation_in_market(Asset_Classes,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,SPY,EEM,EFA,EMB,IPE,DBC,GLD,USO,RWO,GII,DBV
SPY,1,0.698345,0.82749,0.358214,-0.0446688,0.380927,0.0520795,0.340456,0.647125,0.65213,0.542338
EEM,0.698345,1,0.826363,0.446417,-0.0144538,0.511149,0.24841,0.426088,0.668853,0.702845,0.652808
EFA,0.82749,0.826363,1,0.419446,-0.0169981,0.484388,0.181842,0.4141,0.688253,0.74968,0.601714
EMB,0.358214,0.446417,0.419446,1,0.326321,0.287498,0.152914,0.224975,0.56736,0.463501,0.388985
IPE,-0.0446688,-0.0144538,-0.0169981,0.326321,1,0.0445028,0.205795,-0.0120302,0.135893,0.0976927,-0.0365341
DBC,0.380927,0.511149,0.484388,0.287498,0.0445028,1,0.416989,0.863326,0.440849,0.49717,0.497048
GLD,0.0520795,0.24841,0.181842,0.152914,0.205795,0.416989,1,0.230965,0.134896,0.177739,0.154873
USO,0.340456,0.426088,0.4141,0.224975,-0.0120302,0.863326,0.230965,1,0.376192,0.425959,0.411474
RWO,0.647125,0.668853,0.688253,0.56736,0.135893,0.440849,0.134896,0.376192,1,0.723434,0.628147
GII,0.65213,0.702845,0.74968,0.463501,0.0976927,0.49717,0.177739,0.425959,0.723434,1,0.608606


In [13]:
#correlation matrix for US Sectors
corr,ht = correlation_in_market(US_Sectors,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,IYC,IYE,IYF,IYH,IYJ,IYK,IYM,IYR,IYW,IYZ
IYC,1,0.56157,0.779895,0.664168,0.839055,0.761782,0.741219,0.664833,0.744173,0.6943
IYE,0.56157,1,0.550718,0.523693,0.643904,0.580449,0.743304,0.481468,0.476271,0.492143
IYF,0.779895,0.550718,1,0.624788,0.789721,0.684051,0.693126,0.750267,0.58623,0.636168
IYH,0.664168,0.523693,0.624788,1,0.648212,0.697287,0.580246,0.526829,0.50734,0.523614
IYJ,0.839055,0.643904,0.789721,0.648212,1,0.729116,0.812638,0.679015,0.737854,0.715729
IYK,0.761782,0.580449,0.684051,0.697287,0.729116,1,0.687523,0.647117,0.528951,0.576979
IYM,0.741219,0.743304,0.693126,0.580246,0.812638,0.687523,1,0.622877,0.604146,0.590284
IYR,0.664833,0.481468,0.750267,0.526829,0.679015,0.647117,0.622877,1,0.454406,0.540176
IYW,0.744173,0.476271,0.58623,0.50734,0.737854,0.528951,0.604146,0.454406,1,0.650465
IYZ,0.6943,0.492143,0.636168,0.523614,0.715729,0.576979,0.590284,0.540176,0.650465,1


In [14]:
#correlation matrix for Developed Countries Stock
corr,ht = correlation_in_market(Developed_Countries_Stock,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,EWA,EWU,EWC,EWH,EWL,EWD,EWS,EWJ,EWG,EWI,EWP,EWQ
EWA,1,0.777087,0.781971,0.676147,0.701693,0.713173,0.711947,0.607771,0.734676,0.706788,0.697502,0.75711
EWU,0.777087,1,0.771892,0.69597,0.810023,0.799851,0.707255,0.630595,0.838725,0.798651,0.774604,0.870165
EWC,0.781971,0.771892,1,0.642442,0.687848,0.735652,0.663959,0.595826,0.741213,0.691262,0.669689,0.754404
EWH,0.676147,0.69597,0.642442,1,0.612555,0.655289,0.760827,0.589634,0.676876,0.605762,0.600971,0.680664
EWL,0.701693,0.810023,0.687848,0.612555,1,0.763171,0.623611,0.593973,0.825204,0.759523,0.749569,0.835776
EWD,0.713173,0.799851,0.735652,0.655289,0.763171,1,0.671631,0.61021,0.845787,0.763166,0.754925,0.850575
EWS,0.711947,0.707255,0.663959,0.760827,0.623611,0.671631,1,0.609102,0.690872,0.629962,0.631516,0.689829
EWJ,0.607771,0.630595,0.595826,0.589634,0.593973,0.61021,0.609102,1,0.633565,0.577298,0.566263,0.639897
EWG,0.734676,0.838725,0.741213,0.676876,0.825204,0.845787,0.690872,0.633565,1,0.854788,0.831045,0.929373
EWI,0.706788,0.798651,0.691262,0.605762,0.759523,0.763166,0.629962,0.577298,0.854788,1,0.878326,0.896259


In [15]:
#correlation matrix for Emerging Markets Stock
corr,ht = correlation_in_market(Emerging_Markets_Stock,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,EWZ,EWW,EZA,EWY,GXC
EWZ,1,0.659941,0.64851,0.614208,0.56215
EWW,0.659941,1,0.643125,0.653916,0.561321
EZA,0.64851,0.643125,1,0.616482,0.669414
EWY,0.614208,0.653916,0.616482,1,0.598562
GXC,0.56215,0.561321,0.669414,0.598562,1


In [16]:
#correlation matrix for Fixed Income
corr,ht = correlation_in_market(Fixed_Income,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,SHY,IEI,IEF,TLH,TLT,BWX,CRED,JNK,LQD
SHY,1,0.710501,0.754316,0.524396,0.570635,0.239522,0.289473,-0.176164,0.390647
IEI,0.710501,1,0.818252,0.870867,0.70196,0.348643,0.537748,-0.207787,0.417792
IEF,0.754316,0.818252,1,0.845518,0.906877,0.299522,0.52739,-0.166533,0.613659
TLH,0.524396,0.870867,0.845518,1,0.873995,0.289711,0.586575,-0.216764,0.478267
TLT,0.570635,0.70196,0.906877,0.873995,1,0.205456,0.49804,-0.224713,0.568465
BWX,0.239522,0.348643,0.299522,0.289711,0.205456,1,0.447423,0.349165,0.359896
CRED,0.289473,0.537748,0.52739,0.586575,0.49804,0.447423,1,0.389732,0.702835
JNK,-0.176164,-0.207787,-0.166533,-0.216764,-0.224713,0.349165,0.389732,1,0.327466
LQD,0.390647,0.417792,0.613659,0.478267,0.568465,0.359896,0.702835,0.327466,1


In [17]:
#correlation matrix for FX
corr,ht = correlation_in_market(FX,start,end)
corr.style.background_gradient(cmap='coolwarm',axis=None)

,UUP,BZF,FXA,FXB,FXC,FXF,FXS,FXSG,FXY
UUP,1,-0.38112,-0.594625,-0.668674,-0.564779,-0.695184,-0.787272,-0.220862,-0.340272
BZF,-0.38112,1,0.568307,0.332882,0.477831,0.244782,0.434134,0.174867,-0.0714664
FXA,-0.594625,0.568307,1,0.527672,0.720816,0.358109,0.612522,0.209673,-0.0582957
FXB,-0.668674,0.332882,0.527672,1,0.500635,0.411488,0.548841,0.170708,0.00905569
FXC,-0.564779,0.477831,0.720816,0.500635,1,0.297709,0.539094,0.211978,-0.0681669
FXF,-0.695184,0.244782,0.358109,0.411488,0.297709,1,0.517997,0.175717,0.335719
FXS,-0.787272,0.434134,0.612522,0.548841,0.539094,0.517997,1,0.171518,0.103492
FXSG,-0.220862,0.174867,0.209673,0.170708,0.211978,0.175717,0.171518,1,0.154522
FXY,-0.340272,-0.0714664,-0.0582957,0.00905569,-0.0681669,0.335719,0.103492,0.154522,1


In [3]:
app = JupyterDash('abcd')
app.layout = html.Div([
    dcc.RadioItems(
        id='Index Type',
        options=[
            {'label':'Turbulence','value':'Turbulence'},
            {'label':'Absorption Ratio','value':'Absorption Ratio'},
            {'label':'Bar plot','value':'Bar'}
        ],
        value = 'Turbulence'
    ),
    dcc.Graph(id='TB_AR'),
    dcc.Dropdown(
        id='Turbulence',
        options=[
            {'label':'Asset Classes','value':'Asset_Classes'},
            {'label':'US Sectors','value':'US_Sectors'},
            {'label':'Developed Countries Stock','value':'Developed_Countries_Stock'},
            {'label':'Emerging Markets Stock','value':'Emerging_Markets_Stock'},
            {'label':'Fixed Income','value':'Fixed_Income'},
            {'label':'FX','value':'FX'}
        ],
        value='Asset_Classes'
    )
])
    

@app.callback(
    Output('TB_AR', 'figure'),
    [Input('Index Type','value'),
     Input('Turbulence', 'value')])

def update_graph(index_type,market_name):
    if index_type == 'Turbulence':
        traces = []
        traces.append(go.Scatter(
            x=dat1['date'],
            y=dat1[market_name].values,
            opacity=0.7,
            mode = 'lines',
            marker = dict(size=16,
                          color='red'),
            name='Turbulence'
            ))
        layout = go.Layout(
            title='Turbulence'
        )
    elif index_type =='Absorption Ratio':
        
        traces = []
        traces.append(go.Scatter(
            x=dat2['date'],
            y=dat2['AbsorptionRatio'].values,
            opacity=0.7,
            mode = 'lines',
            marker = dict(size=16,
                          color='blue'),
            name='Absorption Ratio'
            ))
        layout = go.Layout(
            title='Absorption Ratio'
        )
        
    else:
        col_list = list(dat1.columns)
        col_list.remove('date')
        bar_list = dat1.iloc[-1,:].values[1:].tolist()
        traces= [go.Bar(
            x=col_list, 
            y=bar_list,
            name = 'Bar Plot'
        )]
        layout = go.Layout(
            title='Bar Plot for Last Day'
        )
    return {
        'data': traces,
        'layout': layout
    }

In [4]:
app